<a href="https://colab.research.google.com/github/iCarrin/WelcomeBike-DC/blob/main/starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np#Isaiah import
from tensorflow import keras
from tensorflow.keras import layers, callbacks

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [ ]:
bikes

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
112470,10/31/2023,19.0,241,594,10.2,10.2,0.4516,8.4,2,4,0,1
112471,10/31/2023,20.0,171,450,8.9,7.5,0.5268,9.3,1,4,0,1
112472,10/31/2023,21.0,176,402,7.7,6.4,0.5756,7.8,1,4,0,1
112473,10/31/2023,22.0,106,257,8.0,8.0,0.5604,0.0,1,4,0,1


In [ ]:
bikes['dteday'] = pd.to_datetime(bikes['dteday'])

In [ ]:
#Isaiah's code
import calendar
# import holidays

# us_holidays = holidays.UnitedStates()

# def find_holiday(row):
#     return int(row['dteday'].date() in us_holidays)

# Define a function to get number of days in a month
def get_days_in_month(row):
    year =  row['dteday'].year     # or 'datetime'.year if using a datetime column
    month = row['dteday'].month
    return calendar.monthrange(year, month)[1]



In [ ]:
# Create target variable
bikes['total_rentals'] = bikes['casual'] + bikes['registered']

# Filter data up to October 2023
train_data = bikes[bikes['dteday'] <= '2023-10-31']

# Define preprocessing function
def preprocess_data(df, scaler=None, fit_scaler=True):
    df = df.copy()
    #Isaiah's added code
    df['days_in_month'] = df.apply(get_days_in_month, axis=1)
    # df['holiday'] = df.apply(find_holiday, axis=1)

    # df['hr'] = df['hr'].astype(int)
    # df['hum'] = np.round(df['hum'], 3)
    df = df.assign(
      year = df['dteday'].dt.year,
      weekend = ((df['workingday'] == 0) & (df['holiday'] == 0)).astype(int),

      hr_sin = np.sin(2 * np.pi * df['hr'] / 24),
      hr_cos = np.cos(2 * np.pi * df['hr'] / 24),
      month_day_sin = np.sin(2 * np.pi * df['dteday'].dt.day / df['days_in_month']),
      month_day_cos = np.cos(2 * np.pi * df['dteday'].dt.day / df['days_in_month']),
      month_sin = np.sin(2 * np.pi * df['dteday'].dt.month / 12),
      month_cos = np.cos(2 * np.pi * df['dteday'].dt.month / 12),

    )
    # Extract day of week
    df['day_of_week'] = df['dteday'].dt.dayofweek
    df.drop(columns=['hr', 'days_in_month'], inplace=True)
    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=['season', 'day_of_week', 'weathersit'], drop_first=True).astype(int)
    # Scale numerical features
    numerical_cols = ['temp_c', 'feels_like_c',  'windspeed']

    if fit_scaler:
        scaler = MinMaxScaler()
        df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    else:
        df[numerical_cols] = scaler.transform(df[numerical_cols])


    return df, scaler

In [ ]:



# Apply preprocessing to training data
train_data, scaler = preprocess_data(train_data, fit_scaler=True)

# Define features and target for training
X_train_full = train_data.drop(['total_rentals', 'casual', 'registered', 'dteday'], axis=1)
y_train_full = train_data['total_rentals']

# Train-test split
# X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

#splitting based on time instead of randomly
split_index = int(len(train_data) * 0.8)#this boosted the mini by 30%
X_train = X_train_full.iloc[:split_index]
y_train = y_train_full.iloc[:split_index]
X_val = X_train_full.iloc[split_index:]
y_val = y_train_full.iloc[split_index:]
print("Training shape:", X_train.shape)
print("Validation shape:", X_val.shape)

Training shape: (89980, 26)
Validation shape: (22495, 26)


In [ ]:
train_data

,dteday,casual,registered,temp_c,feels_like_c,hum,windspeed,holiday,workingday,total_rentals,...,season_4,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,weathersit_2,weathersit_3,weathersit_4
0,1293840000000000000,3,13,0.314815,0.369863,0,0.000000,0,0,16,...,0,0,0,0,0,1,0,0,0,0
1,1293840000000000000,8,30,0.277778,0.342466,0,0.000000,0,0,38,...,0,0,0,0,0,1,0,0,0,0
2,1293840000000000000,5,26,0.277778,0.342466,0,0.014493,0,0,31,...,0,0,0,0,0,1,0,0,0,0
3,1293840000000000000,3,9,0.296296,0.356164,0,0.000000,0,0,12,...,0,0,0,0,0,1,0,0,0,0
4,1293840000000000000,0,1,0.296296,0.356164,0,0.014493,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112470,1698710400000000000,241,594,0.444444,0.465753,0,0.115942,0,1,835,...,1,1,0,0,0,0,0,1,0,0
112471,1698710400000000000,171,450,0.407407,0.424658,0,0.130435,0,1,621,...,1,1,0,0,0,0,0,0,0,0
112472,1698710400000000000,176,402,0.388889,0.410959,0,0.101449,0,1,578,...,1,1,0,0,0,0,0,0,0,0
112473,1698710400000000000,106,257,0.407407,0.438356,0,0.000000,0,1,363,...,1,1,0,0,0,0,0,0,0,0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import Input#changed

# Build model
model = Sequential([
    Input(shape=(X_train.shape[1],)),#changed. It actualy boosted by .02%
    Dropout(.1),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear'),
])


# opt = keras.optimizers.Adam(learning_rate=0.001)

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=28, callbacks=[early_stopping], verbose=1, shuffle=False)

Epoch 1/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 80491.1641 - mse: 80491.1641 - val_loss: 234010.3125 - val_mse: 234010.3125
Epoch 2/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 70005.9844 - mse: 70005.9844 - val_loss: 227652.0156 - val_mse: 227652.0156
Epoch 3/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 68877.6719 - mse: 68877.6719 - val_loss: 209123.9219 - val_mse: 209123.9219
Epoch 4/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 67716.9453 - mse: 67716.9453 - val_loss: 198499.3906 - val_mse: 198499.3906
Epoch 5/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 67375.2266 - mse: 67375.2266 - val_loss: 188890.2188 - val_mse: 188890.2188
Epoch 6/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 65649.3281 - mse: 65649.3281 - val_loss: 178757.3125 - val_mse: 178757.3125
Epoch 7/200
3214/3214 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 65604.7656 - mse: 65604.7656 - val_loss: 175497.8906 - val_mse: 175497.8906
Epoch 8/200
3214/3214 ━

In [ ]:
hist = pd.DataFrame(history.history)
hist = hist.reset_index()
import matplotlib.pyplot as plt
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error ')
    plt.plot(hist['index'], hist['mse'], label='Train Error')
    plt.plot(hist['index'], hist['val_mse'], label = 'Val Error')
    plt.legend()
    # plt.ylim([0,50])

plot_history()

In [ ]:
y_val_pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
rmse = root_mean_squared_error(y_val, y_val_pred)

print(f"Validation MSE: {mse:.2f}")
print(f"Validation MAE: {mae:.2f}")
print(f"Validation R²: {r2:.2f}")
print(f"Validation RMSE: {rmse:.2f}")
# Validation MSE: 23791.23
# Validation MAE: 102.14
# Validation R²: 0.83 (-.08)
# Validation RMSE: 154.24

In [ ]:
#save the training order
column_order = X_train.columns.tolist()
#import the mini
bikes_mini = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
#clean up the data the way you've been cleaning
bikes_mini['dteday'] = pd.to_datetime(bikes_mini['dteday'])
mini_clean, _ = preprocess_data(bikes_mini, scaler, False)

#add the weather and season that the mini set is missing as a column of 0's
cols_to_add = ['weathersit_1', 'weathersit_2', 'weathersit_3', 'weathersit_4', 'season_1', 'season_2', 'season_3', 'season_4']
for col in cols_to_add:
  if col not in mini_clean.columns:
    mini_clean[col] = 0
#Orginize them the way the original had
mini_clean = mini_clean[column_order]
#run it against your modle
predictions = model.predict(mini_clean)

#make the preditions the right format
output = pd.DataFrame(predictions)
output.columns = ['predictions']
#export
output.to_csv('karen13.6-predictions.csv', index=False)
print("Predictions saved to predictions.csv")